Librairies

In [35]:
import os
import numpy as np
import pandas as pd
import cv2

from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from skimage.metrics import structural_similarity as ssim

Metriques

In [65]:
# Ouvrir les images
def img_open(path_sal_map,path_attacked_sal_map):
    img1 = cv2.imread(path_sal_map)
    img2 = cv2.imread(path_attacked_sal_map)
    return img1,img2

# Calcul de la différence absolue
# Plus la valeur est proche de 0 plus les images sont similaires
def diff_abs(path_sal_map,path_attacked_sal_map):
    img1 = cv2.imread(path_sal_map)
    img2 = cv2.imread(path_attacked_sal_map)
    return (np.sum(np.abs(img1 - img2)))/(img1.size)

# Calcul de la différence quadratique (mse)
# Plus la valeur est proche de 0 plus les images sont similaires
def diff_quadratique(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    diff = cv2.subtract(img1, img2)
    error = np.sum(np.square(diff))
    return error/(img1.size)


# Calcul du coef de corrélation
# Plus la valeur est proche de 1 plus les images sont similaires
def coef_correlation(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    result = pearsonr(img1.flatten(),img2.flatten())
    return result.statistic

# Calcul du ssim Structural Similarity Index
# Plus la valeur est proche de 1 plus les images sont similaires
def ssim_func(path_sal_map,path_attacked_sal_map):
    img1, img2 = img_open(path_sal_map,path_attacked_sal_map)
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    valssim = ssim(gray_img1, gray_img2)
    return valssim

In [59]:
path_project_root = os.path.dirname(os.path.abspath(''))
directory = os.path.join(path_project_root, 'final_dir\images\output')

object = []
model = []
attack = []
intensity= []
saliency = []
attacked = []
path = []

for file in os.listdir(directory):
    filename = file[:-4]
    info = filename.split('_')
    object.append(info[0])
    model.append(info[1])
    attack.append(info[2])
    intensity.append(info[3])
    saliency.append(info[4])
    attacked.append(info[5])
    path.append(os.path.join(directory, filename))
   
df = pd.DataFrame(list(zip(object,model,attack,intensity,saliency,attacked,path)), 
                  columns = ['Objet','Model','Attaque','Intensité','Saillance','Attaquée','Chemin'])


df_saliency_attacked = df[(df['Saillance'] == 'saliency') & (df['Attaquée'] == 'attacked')].reset_index()
df_saliency_noattacked = df[(df['Saillance'] == 'saliency') & (df['Attaquée'] == 'noattacked')].reset_index()

df_img_attacked = df[(df['Saillance'] == 'nosaliency') & (df['Attaquée'] == 'attacked')].reset_index()
df_img_noattacked = df[(df['Saillance'] == 'nosaliency') & (df['Attaquée'] == 'noattacked')].reset_index()


In [ ]:
diff_abs_result = []

for i in df_saliency_noattacked.index:
    diff_abs_result.append(diff_quadratique(df_saliency_noattacked["Chemin"][i],df_saliency_attacked["Chemin"][i]))